Factor calculation

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sql_functions import get_dataframe

In [5]:
#Get the data from sql
schema = 'capstone_travel_index'
airbnb_df = get_dataframe(f'SELECT * FROM {schema}.airbnb_prices_all')

In [8]:
airbnb_df.head()

,Entire home/apt,Hotel room,Private room,Shared room,city
0,248.898913,177.322581,150.766495,108.230769,Amsterdam
1,126.555447,148.000000,90.354839,65.000000,Antwerp
2,164.000244,404.632800,124.963620,59.880229,Asheville
3,95.573357,331.087591,102.103448,39.728395,Athens
4,310.408633,720.033600,161.581835,65.029578,Austin


In [7]:
#Get the data from sql
schema = 'capstone_travel_index'
numbeo_df = get_dataframe(f'SELECT * FROM {schema}.city_apart_prices')

In [9]:
numbeo_df.head()

,average_price,item_name,city,country,price_per_day,calc_price
0,561.444444,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,18.714815,74.859259
1,726.963403,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,Denmark,24.232113,96.928454
2,640.494990,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,United Kingdom,21.349833,85.399332
3,491.224523,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,Ivory Coast,16.374151,65.496603
4,674.859546,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,Ghana,22.495318,89.981273


In [10]:
#drop the columns
airbnb_df = airbnb_df.drop(['Hotel room', 'Private room', 'Shared room'], axis = 1)

In [11]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entire home/apt  86 non-null     float64
 1   city             86 non-null     object 
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


In [12]:
#drop the columns
numbeo_df = numbeo_df.drop(['average_price', 'calc_price', 'item_name'], axis = 1)

In [13]:
numbeo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           524 non-null    object 
 1   country        524 non-null    object 
 2   price_per_day  524 non-null    float64
dtypes: float64(1), object(2)
memory usage: 12.4+ KB


In [14]:
numbeo_df['city'].nunique()

519

In [15]:
#merge airbnb and numbeo

merged_df = pd.merge(airbnb_df, numbeo_df, on='city')

In [20]:
merged_df.head(50)

,Entire home/apt,city,country,price_per_day
0,248.898913,Amsterdam,Netherlands,55.077957
1,126.555447,Antwerp,Belgium,28.070175
2,95.573357,Athens,Greece,37.320684
3,310.408633,Austin,United States,71.524476
4,77.920512,Bangkok,Thailand,20.219194
5,180.903836,Barcelona,Spain,35.945185
6,128.786118,Berlin,Germany,43.294949
7,143.796868,Bergamo,Italy,21.111111
8,254.140615,Bologna,Italy,26.297619
9,215.610694,Boston,United States,86.008842


In [24]:
merged_df['factor'] = merged_df['Entire home/apt'] / merged_df['price_per_day']

In [25]:
merged_df.head(50)

,Entire home/apt,city,country,price_per_day,factor
0,248.898913,Amsterdam,Netherlands,55.077957,4.519030
1,126.555447,Antwerp,Belgium,28.070175,4.508538
2,95.573357,Athens,Greece,37.320684,2.560868
3,310.408633,Austin,United States,71.524476,4.339894
4,77.920512,Bangkok,Thailand,20.219194,3.853789
5,180.903836,Barcelona,Spain,35.945185,5.032770
6,128.786118,Berlin,Germany,43.294949,2.974622
7,143.796868,Bergamo,Italy,21.111111,6.811431
8,254.140615,Bologna,Italy,26.297619,9.664016
9,215.610694,Boston,United States,86.008842,2.506843


In [26]:
merged_df.describe()

,Entire home/apt,price_per_day,factor
count,70.000000,70.000000,70.000000
mean,178.242192,47.482783,4.410715
std,59.696938,24.045550,2.184770
min,58.021089,10.738235,1.999417
25%,132.007833,28.279147,3.055562
50%,174.910400,43.748030,3.932473
75%,217.527752,63.462004,4.815151
max,315.965080,114.092501,16.154458


In [28]:
merged_df.sort_values('factor')

,Entire home/apt,city,country,price_per_day,factor
68,228.118510,New York,United States,114.092501,1.999417
52,160.663079,Seattle,United States,76.690769,2.094947
54,242.838383,Singapore,Singapore,112.025842,2.167700
24,149.992455,Hong Kong,Hong Kong (China),68.850463,2.178525
44,126.555052,Portland,United States,54.850445,2.307275
...,...,...,...,...,...
22,232.357377,Florence,Italy,29.102564,7.984086
33,223.570569,Malaga,Spain,26.570327,8.414295
13,182.459755,Cape Town,South Africa,19.622982,9.298269
8,254.140615,Bologna,Italy,26.297619,9.664016


In [30]:
# Display the entire DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
merged_df.sort_values('factor')

,Entire home/apt,city,country,price_per_day,factor
68,228.118510,New York,United States,114.092501,1.999417
52,160.663079,Seattle,United States,76.690769,2.094947
54,242.838383,Singapore,Singapore,112.025842,2.167700
24,149.992455,Hong Kong,Hong Kong (China),68.850463,2.178525
44,126.555052,Portland,United States,54.850445,2.307275
51,248.651916,San Francisco,United States,101.183293,2.457440
9,215.610694,Boston,United States,86.008842,2.506843
69,190.275729,Washington,United States,75.124256,2.532814
2,95.573357,Athens,Greece,37.320684,2.560868
65,205.515607,Zurich,Switzerland,80.238849,2.561298


In [34]:
# Outlier Rio de Janeiro drop
merged_df.drop(47).describe()

,Entire home/apt,price_per_day,factor
count,69.000000,69.000000,69.000000
mean,177.824109,47.985151,4.240515
std,60.030964,23.848800,1.669087
min,58.021089,10.738235,1.999417
25%,131.951993,28.906061,3.047376
50%,174.666805,44.201111,3.890674
75%,217.832603,63.500565,4.784730
max,315.965080,114.092501,9.664016


In [ ]:
# Without Outlier Rio de Janeiro
# Median is 3.89
# Mean/avg is 4.24
# For calc_price we decide to take factor 3.89 